# 0.사전준비

### google 스프레드 시트 공유문서를 사용하기 위한 서비스 가입FLOW

[구글API발급]
1. 구글로그인진행
2. https://console.cloud.google.com/apis 접속
3. 상단 프로젝트 선택 클릭
4. 프로젝트 추가 , 이름은 "bhyunco-cowork" 라고 설정
5. 우측 "사용자 인증 정보" 클릭
6. 상단의 "CREATE CREDENTIALS" 클릭 > "서비스 계정" 클릭
7. 서비스 계정 이름 "bhyunco-cowork" 입력 그리고 전체 Enter
8. 서비스 계정 생성 후 "세부정보" 탭 클릭 > 이메일,고유 ID 기록하기
9. 서비스 계정의 '키' 탭 클릭 > '키 추가' 클릭 > '새 키 만들기' 클릭 > 'json' 선택 후 만들기
10. 자동다운로드 된 json 파일을 보관하기
11. 검색 google sheets API 사용설정
12. 검색 google drive API 사용설정

[구글 드라이브에서 스프레드 시트 만들고 공유]
13. google 드라이브에서 구글스프레드 시트를 만들고 'bhyunco-cowork-gspread' 라는 이름으로 설정
14. 구글스프레드 시트의 공유버튼을 눌러서 8번에서 가져온 이메일 추가하기
15. .json 파일을 구글드라이브에 넣어두고 공유링크 만들고 아래의 file_id 에 id부분만 적어넣기(추후 다운로드)

[코드로 설정]
16. 아래코드 실행

##### PC (jupyter notebook) 활용 

Colab 활용

In [1]:
#파일다운로드
#colab용
!pip install gdown==4.5.1
import gdown
import getpass
google_path = "https://drive.google.com/uc?id="
drive_url = "https://drive.google.com/file/d/1wJ7CmStuzHhgwc5-giLns5XzBMZykJ-I/view?usp=sharing"#"구글드라이브 json파일 URL경로"
file_id = drive_url.split('/')[-2]  #.json 파일 경로에서 id 추출 ex - https://drive.google.com/file/????????????/view?usp=sharing  --> 이거라면 ???????????? 로 저장
gdown.download(google_path+file_id,'test.json',quiet=False)
output_name = 'test.json'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14949 sha256=6c833ef95c931f247da637feba7b3d4ca23bcf41099f7ee32ca3878174f2329e
  Stored in directory: /root/.cache/pip/wheels/8d/df/71/846b2aa0fabaac2af23fbc5214eeaa55f0616e9d1a05187d72
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


Downloading...
From: https://drive.google.com/uc?id=1wJ7CmStuzHhgwc5-giLns5XzBMZykJ-I
To: /content/test.json
100%|██████████| 2.35k/2.35k [00:00<00:00, 1.60MB/s]


In [2]:
import os

file_list = os.listdir()
print(file_list)
for file in file_list:
    if ".json" in file:
        output_name = file

['.config', 'test.json', 'sample_data']


In [3]:
# gspread 에서 받은 데이터를 데이터프레임으로 변경하는 함수
def gsheetdf(sheet):
    df = pd.DataFrame(columns=list(sheet.get_all_records()[0].keys()))
    for item in sheet.get_all_records():
        df.loc[len(df)] = item
    return df

In [4]:
#구글스프레드시트 - 파이썬 연동시키기

!pip install gspread==5.4.0
!pip install --upgrade oauth2client
from oauth2client.service_account import ServiceAccountCredentials
import gspread
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(output_name, scope)
spreadsheet_name = "bhyunco-cowork-gspread" #스프레드 시트 이름
client = gspread.authorize(creds)
spreadsheet = client.open(spreadsheet_name)
## 시트이름에 따른 변수 설정 by name
# sheet = spreadsheet.worksheet("시트1")
# print(sheet)
# ## OR by index
sheet = spreadsheet.get_worksheet(0)
print(sheet)
# 데이터 가져오기
sheet.get_all_records()
# pandas로 변경하기
import pandas as pd
df_sample_data = gsheetdf(sheet)
df_sample_data.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
<Worksheet '1231' id:0>


,1,b1 updated,3,
0,4,5,6,
1,7,8,9,
2,data1,data2,data3,data4
3,data5,data6,data7,data8


# 1.현업적용코드보기

> 설문 URL = https://forms.gle/MXxRsW8GFCdsEV2T6



In [6]:
#구글스프레드시트 - 파이썬 연동시키기
!pip install gspread_dataframe==3.3.0
!pip install gspread==5.4.0
!pip install --upgrade oauth2client
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(output_name, scope)
spreadsheet_name = "bhyunco_gp_result" #스프레드 시트 이름

client = gspread.authorize(creds)
spreadsheet = client.open(spreadsheet_name)
## 시트이름에 따른 변수 설정 by name
#3. 해당 데이터를 원하는 형태로 가공하는 프로그램을 제작한다.

# 응답지가 있는 데이터 시트를 설정
survey_sheet = spreadsheet.get_worksheet(0)
#불러온 dict 데이터를 pandas DataFrame으로 만들기
import pandas as pd

df = gsheetdf(survey_sheet)
#df 컬럼값 변경
df.rename(columns = {'타임스탬프':'Time','사번':'epnum','일정':'class'},inplace=True)
# 마지막만 남기기
df = df.drop_duplicates(['epnum'], keep = 'last')
# 클래스별 현황표 출력
result_df = df.pivot_table(index='class',values='epnum',aggfunc='count')
display(result_df)
result_df['class'] = result_df.index
result_df = result_df[['class','epnum']]
sheet_unit = spreadsheet.get_worksheet(1)
# 기존에 있던 데이터를 모두 삭제
sheet_unit.clear()
# 만들어진 result_df 를 원하는 시트에 집어넣기
import gspread_dataframe as gd
gd.set_with_dataframe(sheet_unit, result_df)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyError: ignored

# 2.co_work 기본문법

In [ ]:
# 폴더안에 있는 json 파일 가져오기
import os

file_list = os.listdir()
print(file_list)
for file in file_list:
    if ".json" in file:
        output_name = file

['.config', 'test.json', 'sample_data']


In [ ]:
# gspread 에서 받은 데이터를 데이터프레임으로 변경하는 함수
def gsheetdf(sheet):
    df = pd.DataFrame(columns=list(sheet.get_all_records()[0].keys()))
    for item in sheet.get_all_records():
        df.loc[len(df)] = item
    return df

In [ ]:
    #구글스프레드시트 - 파이썬 연동시키기
!pip install gspread==5.4.0
!pip install --upgrade oauth2client
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(output_name, scope)
spreadsheet_name = "bhyunco-cowork-gspread" #스프레드 시트 이름
client = gspread.authorize(creds)
spreadsheet = client.open(spreadsheet_name)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#스프레드시트의 세부 시트 확인하기
spreadsheet.worksheets()

[<Worksheet '1231' id:0>]

In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#스프레드시트의 세부 시트 중 원하는 시트를 변수로 설정하기
sheet_unit = spreadsheet.worksheet('1231')
print(sheet_unit)
# ## OR by index
sheet = spreadsheet.get_worksheet(0)
print(sheet)

<Worksheet '1231' id:0>
<Worksheet '1231' id:0>


In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#스프레드시트의 세부 시트 데이터 불러오기
sheet_unit.get_all_records()

[{'1': 4, '2': 5, '3': 6}, {'1': 7, '2': 8, '3': 9}]

In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#불러온 dict 데이터를 pandas DataFrame으로 만들기
import pandas as pd
def gsheetdf(sheet):
    df = pd.DataFrame(columns=list(sheet.get_all_records()[0].keys()))
    for item in sheet.get_all_records():
        df.loc[len(df)] = item
    return df
df_sample_data = gsheetdf(sheet)
df_sample_data.head()

,1,2,3
0,4,5,6
1,7,8,9


In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#스프레드 시트 데이터 작성하기

#cell update
sheet_unit.update_acell('B1','b1 updated')

{'spreadsheetId': '1iubx-obLFd8qBh1OoG31CEMGYAFZ_RyNOMs_l3mV0rY',
 'updatedRange': "'1231'!B1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#행 추가하기
sheet_unit.append_row(['data1', 'data2', 'data3', 'data4'])

{'spreadsheetId': '1iubx-obLFd8qBh1OoG31CEMGYAFZ_RyNOMs_l3mV0rY',
 'tableRange': "'1231'!A1:C3",
 'updates': {'spreadsheetId': '1iubx-obLFd8qBh1OoG31CEMGYAFZ_RyNOMs_l3mV0rY',
  'updatedRange': "'1231'!A4:D4",
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [ ]:
#write your code (백견이 불여일타)

In [ ]:
#특정 행 추가하기
sheet_unit.insert_row(['data5', 'data6', 'data7', 'data8'], 5)

{'spreadsheetId': '1iubx-obLFd8qBh1OoG31CEMGYAFZ_RyNOMs_l3mV0rY',
 'updates': {'spreadsheetId': '1iubx-obLFd8qBh1OoG31CEMGYAFZ_RyNOMs_l3mV0rY',
  'updatedRange': "'1231'!A5:D5",
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [ ]:
#write your code (백견이 불여일타)

# 3.현업적용사례

### 코드설계노트

1. 승진교육을 선택하게 된 사람들이 교육참여 일정 체크한 설문파일을 만든다.
2. python으로 google spread 시트로 연동한다.
3. 해당 데이터를 원하는 형태로 가공하는 프로그램을 제작한다.
4. 완성된 구글 시트를 게시판에 올린다.

In [ ]:
#1. 승진교육을 선택하게 된 사람들이 교육참여 일정 체크한 설문파일을 만든다.

#설문파일제작방법 실습에서는 수동으로 작업한다.
# 만약 진행해보고 싶다면, 설문지를 만들어서 이미 만든 gspared-test3와 연동시킨다.

In [ ]:
#write your code()

In [ ]:
#2. python으로 google spread 시트로 연동한다.

#python 연동 작업 위에서 완료하였음

In [ ]:
#write your code()

In [ ]:
#3. 해당 데이터를 원하는 형태로 가공하는 프로그램을 제작한다.
#스프레드 시트 세팅
    #구글스프레드시트 - 파이썬 연동시키기
!pip install gspread==5.4.0
!pip install --upgrade oauth2client
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(output_name, scope)
spreadsheet_name = "bhyunco_gp_result" #스프레드 시트 이름
client = gspread.authorize(creds)
spreadsheet = client.open(spreadsheet_name)
#시트명 확인하기
spreadsheet.worksheets()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[<Worksheet '설문지응답' id:811236768>,
 <Worksheet '시트1' id:0>,
 <Worksheet '설문응답요약' id:1221342578>,
 <Worksheet '시트3' id:1199452600>,
 <Worksheet '시트4' id:1467118139>]

In [ ]:
#write your code()

In [ ]:

# 응답지가 있는 데이터 시트를 설정
survey_sheet = spreadsheet.get_worksheet(0)
print(survey_sheet)

<Worksheet '설문지응답' id:811236768>


In [ ]:
#write your code()

In [ ]:
#불러온 dict 데이터를 pandas DataFrame으로 만들기
import pandas as pd
def gsheetdf(sheet):
    df = pd.DataFrame(columns=list(sheet.get_all_records()[0].keys()))
    for item in sheet.get_all_records():
        df.loc[len(df)] = item
    return df
df = gsheetdf(survey_sheet)
df.head()


,타임스탬프,사번,일정
0,2022. 8. 8 오후 20:36:21,1,3차-2030.09.04
1,2022. 8. 8 오후 20:36:22,30,4차-2030.09.05
2,2022. 8. 8 오후 20:36:23,31,1차-2030.09.02
3,2022. 8. 8 오후 20:36:24,32,2차-2030.09.03
4,2022. 8. 8 오후 20:36:25,33,4차-2030.09.05


In [ ]:
#write your code()

In [ ]:
#df 컬럼값 변경
df.rename(columns = {'타임스탬프':'Time','사번':'epnum','일정':'class'},inplace=True)
print(df.shape)

(68, 3)


In [ ]:
#write your code()

In [ ]:
#각 사번 최종 작성 데이터를 최종본으로 변경
pdf = df.pivot_table(index='epnum',values='class',aggfunc='count')
display(pdf[pdf['class']>1])
print(pdf.shape)

,class
epnum,
2,2
5,3
25,2
398,2


(63, 1)


In [ ]:
#write your code()

In [ ]:
# 마지막만 남기기
print(df.shape)
df = df.drop_duplicates(['epnum'], keep = 'last')
print(df.shape)

(68, 3)
(63, 3)


In [ ]:
#write your code()

In [ ]:
# 클래스별 현황표 출력
result_df = df.pivot_table(index='class',values='epnum',aggfunc='count')
display(result_df)
result_df['class'] = result_df.index
result_df = result_df[['class','epnum']]
display(result_df)

,epnum
class,
1차-2030.09.02,18
2차-2030.09.03,7
3차-2030.09.04,13
4차-2030.09.05,18
5차-2030.09.06,7


,class,epnum
class,,
1차-2030.09.02,1차-2030.09.02,18
2차-2030.09.03,2차-2030.09.03,7
3차-2030.09.04,3차-2030.09.04,13
4차-2030.09.05,4차-2030.09.05,18
5차-2030.09.06,5차-2030.09.06,7


In [ ]:
#write your code()

In [ ]:
#설문지응답요약SHEET 로 sheet_unit 변수설정하기
sheet_unit = spreadsheet.get_worksheet(1)
sheet_unit

<Worksheet '시트1' id:0>

In [ ]:
#write your code()

In [ ]:
# 기존에 있던 데이터를 모두 삭제
sheet_unit.clear()

{'spreadsheetId': '177K95nATSk_21TGCIiQXZ81USqdYeDkwe7xW2uBcXL8',
 'clearedRange': "'시트1'!A1:Z1004"}

In [ ]:
#write your code()

In [ ]:
# 만들어진 result_df 를 원하는 시트에 집어넣기
import gspread_dataframe as gd
gd.set_with_dataframe(sheet_unit, result_df)

In [ ]:
#write your code()

In [ ]:
# 4. 완성된 구글 시트를 게시판에 올린다.